all exercises based on [Introduction to Deep Learning for NLP](https://wikidocs.net/31766)

텍스트를 숫자(정수)로 표현하는 것은 원핫인코딩/워드임베딩 등과 관련 있음. 실제로 어떤 과정으로 단어 텍스트에 정수 인덱스를 부여할 수 있는지 정리할 것.<br>

방법 중 하나: 단어를 빈도수 순으로 정렬한 단어 집합에 많은 단어부터 차례로 정수 부여

### dict 사용

In [1]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [4]:
# 문장 토큰화
sent = sent_tokenize(text)

In [25]:
# 정제 작업과 함께
# 단어 토큰화
clean_words = []
stop_words = stopwords.words('english')
# 단어별 빈도수 기록
vocab = {}

for i in sent:
    words = i.lower()
    words = word_tokenize(words)
    clean = []
    for w in words:
        if (w not in stop_words) & (len(w)>2): # 글자수 2 이하는 제외
            clean.append(w)
            if w not in vocab:
                vocab[w] = 1
            else: vocab[w] += 1
    clean_words.append(clean)
print(clean_words)
print(vocab)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [19]:
# 빈도 높은 순 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True) # dict.items는 (key, value)의 튜플로 return
vocab_sorted

[('barber', 8),
 ('secret', 6),
 ('huge', 5),
 ('kept', 4),
 ('person', 3),
 ('word', 2),
 ('keeping', 2),
 ('good', 1),
 ('knew', 1),
 ('driving', 1),
 ('crazy', 1),
 ('went', 1),
 ('mountain', 1)]

In [23]:
# 빈도순 인덱스 부여
word_to_index = {}
for i in range(len(vocab_sorted)):
    if vocab_sorted[i][1]>=2: # 빈도수 2 미만인 건 제외
        word_to_index[vocab_sorted[i][0]] = i+1
word_to_index

{'barber': 1,
 'secret': 2,
 'huge': 3,
 'kept': 4,
 'person': 5,
 'word': 6,
 'keeping': 7}

In [24]:
# 기존의 token들을 전부 인코딩할 건데, 빈도수 때문에 삭제된 단어들이 있음.
# 그 단어들을 하나의 단어 집합으로 생각하고 인덱스 부여 (etc처럼)
word_to_index['OOV'] = len(word_to_index)+1

In [35]:
encoded = []
for i in clean_words:
    # encoded_sent = [ind for j in i for e,ind in word_to_index.items() if e==j]
    encoded_sent = []
    for j in i:
        if j in word_to_index.keys():
            encoded_sent.append(word_to_index[j])
        else:
            encoded_sent.append(word_to_index['OOV'])
    encoded.append(encoded_sent)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [8, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 8, 1, 8], [1, 8, 3, 8]]


### 더 쉽게 Counter 사용

In [36]:
from collections import Counter

In [37]:
print(clean_words)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [39]:
# 위의 단어 집합을 하나로 만들어 줄 것 (내부 리스트 경계 없앨 것)
words = sum(clean_words, []) 
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [46]:
vocab = Counter(words)
print(vocab)
# Counter 객체에 리스트 내 원소의 빈도수가 알아서 저장됨 (Counter - dict 형태)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [47]:
# 등장 빈도 수가 높은 상위 5개 지정
vocab_most = vocab.most_common(5)


In [48]:
vocab_most

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [50]:
# 정수 인덱스 부여
word_to_index = {}
ind = 1
for i, j in vocab_most:
    word_to_index[i] = ind
    ind +=1
word_to_index

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}

### nltk의 FreqDist 사용
Counter와 비슷

In [51]:
from nltk import FreqDist
import numpy as np

In [52]:
# np.hstack으로 문장 구분 제거 후 입력으로 사용 (위에서는 sum(리스트, [])로 했었음)

vocab = FreqDist(np.hstack(clean_words))
print(vocab) # 프린트는 안됨 FreqDist 객체

<FreqDist with 13 samples and 36 outcomes>


In [53]:
print(vocab['barber']) # 인덱싱은 됨

8


In [54]:
vocab_most = vocab.most_common(5) # Counter 객체랑 똑같이 most_common 사용 가능, return 값은 리스트
vocab_most

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [56]:
# 인덱스 부여.
word_to_index = {word[0]: ind+1 for ind, word in enumerate(vocab_most)} # dict comprehension (방식은 list comprehension과 같음)
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


In [61]:
print(enumerate(vocab_most))

### cf. enumerate

순서가 있는 자료형을 입력으로 받아, 인덱스를 순차적으로 함께 리턴

In [64]:
sample = [(1,'one'), (2, 'two'), (3, 'three'), (4, 'four')]
for i in enumerate(sample):
    print(i, sample[i[0]][1])

(0, (1, 'one')) one
(1, (2, 'two')) two
(2, (3, 'three')) three
(3, (4, 'four')) four


### Keras 전처리 도구 사용

In [65]:
from tensorflow.keras.preprocessing.text import Tokenizer
clean_words

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

In [66]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_words) # fit_on_texts 인자로 코퍼스를 입력하면, 빈도수 기준으로 단어 집합 생성
# 이중리스트도 상관 없나봄

In [67]:
tokenizer.word_index # 알아서 빈도순으로 인덱스 부여

{'barber': 1,
 'secret': 2,
 'huge': 3,
 'kept': 4,
 'person': 5,
 'word': 6,
 'keeping': 7,
 'good': 8,
 'knew': 9,
 'driving': 10,
 'crazy': 11,
 'went': 12,
 'mountain': 13}

In [68]:
tokenizer.word_counts # 빈도 확인도 가능. 알아서 저장됨

OrderedDict([('barber', 8),
             ('person', 3),
             ('good', 1),
             ('huge', 5),
             ('knew', 1),
             ('secret', 6),
             ('kept', 4),
             ('word', 2),
             ('keeping', 2),
             ('driving', 1),
             ('crazy', 1),
             ('went', 1),
             ('mountain', 1)])

In [69]:
tokenizer.texts_to_sequences(clean_words) # 이렇게 이미 정해진 index(tokenizer에 들어있는 정보)를 반영해 기존 문장(입력)을 정수 리스트로 변환

[[1, 5],
 [1, 8, 5],
 [1, 3, 5],
 [9, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13]]

In [71]:
# 상위 5개 단어만
tokenizer = Tokenizer(num_words = 5+1) # 상위 5개 단어만 사용. num_words는 0개부터 카운트하기 때문에
tokenizer.fit_on_texts(clean_words)

In [72]:
tokenizer.word_index # num_words 설정했는데도 인덱스는 다 저장됨

{'barber': 1,
 'secret': 2,
 'huge': 3,
 'kept': 4,
 'person': 5,
 'word': 6,
 'keeping': 7,
 'good': 8,
 'knew': 9,
 'driving': 10,
 'crazy': 11,
 'went': 12,
 'mountain': 13}

In [73]:
tokenizer.texts_to_sequences(clean_words) # num_words 설정한 게 반영되어 인덱스 6 이상의 단어들(비교적 빈도가 적은)이 삭제됨

[[1, 5],
 [1, 5],
 [1, 3, 5],
 [2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4],
 [1, 4],
 [1, 4, 2],
 [3, 2, 1],
 [1, 3]]

In [75]:
# 빈도 적은 단어들 OOV로 보존하고 싶다면!
tokenizer = Tokenizer(num_words = 5+2, oov_token = 'OOV')
tokenizer.fit_on_texts(clean_words)

In [76]:
print('OOV의 인덱스: ', tokenizer.word_index['OOV'])

OOV의 인덱스:  1


In [77]:
print(tokenizer.texts_to_sequences(clean_words)) # 빈도 상위 5개는 2~6, OOV는 1로 인덱싱 됨

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]
